In [1]:
import sys
import ROOT
import pyfftw

ROOT.gInterpreter.AddIncludePath('../../include/')
ROOT.gInterpreter.Declare('#include "RoEvent.h"')
ROOT.gSystem.Load('../../libs/libRoEvent.so')

%jsroot on

import pickle
from array import array
import numpy as np
sys.path.append("../package/")
import lycro 


Welcome to JupyROOT 6.24/00


In [4]:
run = "r1177"
thr = 70; pad = 30
cards = [1, 2, 3, 4, 5, 6]
root = "../../tmp/ro_" + run + ".root"
pick = "../../tmp/ju_" + run + "_thr-" + str(thr) + "_pad-" + str(pad) + ".pik"
print("ROOT file: {}".format(root))
print("PICK file: {}".format(pick))

#get data from RoEvent class (to generate it use dq script)
f = ROOT.TFile(root)
myTree = f.Get("data"); myTree.Print(); myTree.Show(560)

#loking signals
ro_signals = []
numEv = myTree.GetEntries()
for entry in myTree:
    events = entry.events
    numCh = events.getNumPayloads()
    ev = events.getHeader().getEvtNum()
    print("Ev {}:{} is processing... ".format(ev, numEv), end='\r')
    for n in range(numCh):
        ch = events.getPayloads().AddrAt(n).getCh()
        cro = events.getPayloads().AddrAt(n).getCRO()
        numSa = events.getPayloads().AddrAt(n).getNsaCRO()
        data = lycro.alignment(cro, 0)
        ev_signals_pos = lycro.findSignals(ev, ch, data, thr, pad, 1)
        ev_signals_neg = lycro.findSignals(ev, ch, data, thr, pad, -1)
        ro_signals.extend(ev_signals_pos)
        ro_signals.extend(ev_signals_neg)

with open(pick, "wb") as f:
    pickle.dump(len(ro_signals), f)
    for value in ro_signals:
        pickle.dump(value, f)

ROOT file: ../../tmp/ro_r1177.root
PICK file: ../../tmp/ju_r1177_thr-70_pad-30.pik
******************************************************************************
*Tree    :data      : data                                                   *
*Entries :     3162 : Total =      8106728905 bytes  File  Size = 1819467027 *
*        :          : Tree compression factor =   4.46                       *
******************************************************************************
*Branch  :events                                                             *
*Entries :     3162 : BranchElement (see below)                              *
*............................................................................*
*Br    0 :fUniqueID : UInt_t                                                 *
*Entries :     3162 : Total  Size=      13229 bytes  File Size  =        163 *
*Baskets :        1 : Basket Size=      32000 bytes  Compression=  78.06     *
*...............................................